In [1]:
from keras import backend as K
from keras.models import load_model
from keras.preprocessing import image
from keras.optimizers import Adam
from imageio import imread
import numpy as np
import matplotlib.patches as patches
from matplotlib import pyplot as plt

from models.keras_ssd300 import ssd_300
from models.keras_ssd512 import ssd_512
from keras_loss_function.keras_ssd_loss import SSDLoss
from keras_layers.keras_layer_AnchorBoxes import AnchorBoxes
from keras_layers.keras_layer_DecodeDetections import DecodeDetections
from keras_layers.keras_layer_DecodeDetectionsFast import DecodeDetectionsFast
from keras_layers.keras_layer_L2Normalization import L2Normalization

from ssd_encoder_decoder.ssd_output_decoder import decode_detections, decode_detections_fast

from data_generator.object_detection_2d_data_generator import DataGenerator
from data_generator.object_detection_2d_photometric_ops import ConvertTo3Channels
from data_generator.object_detection_2d_geometric_ops import Resize
from data_generator.object_detection_2d_misc_utils import apply_inverse_transforms

import tensorflow as tf
import keras as keras

Using TensorFlow backend.
/home/aldopedraza/Documentos/ssd/data_generator/object_detection_2d_data_generator.py:43: UserWarning: 'BeautifulSoup' module is missing. The XML-parser will be unavailable.
  warnings.warn("'BeautifulSoup' module is missing. The XML-parser will be unavailable.")


In [76]:
path_models = '../models'

# Load the trained model

In [77]:
def load_model(model_path, height, width):
    # 1: Build the Keras model
    K.clear_session() # Clear previous models from memory.

    if height == 300:
        model = ssd_300(image_size=(height, width, 3),
                        n_classes=20,
                        mode='inference',
                        l2_regularization=0.0005,
                        scales=[0.1, 0.2, 0.37, 0.54, 0.71, 0.88, 1.05], # The scales for MS COCO are [0.07, 0.15, 0.33, 0.51, 0.69, 0.87, 1.05]
                        aspect_ratios_per_layer=[[1.0, 2.0, 0.5],
                                                 [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                                                 [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                                                 [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                                                 [1.0, 2.0, 0.5],
                                                 [1.0, 2.0, 0.5]],
                        two_boxes_for_ar1=True,
                        steps=[8, 16, 32, 64, 100, 300],
                        offsets=[0.5, 0.5, 0.5, 0.5, 0.5, 0.5],
                        clip_boxes=False,
                        variances=[0.1, 0.1, 0.2, 0.2],
                        normalize_coords=True,
                        subtract_mean=[123, 117, 104],
                        swap_channels=False,
                        confidence_thresh=0.5,
                        iou_threshold=0.45,
                        top_k=200,
                        nms_max_output_size=400)
    elif height == 512:
        model = ssd_512(image_size=(height, width, 3),
                n_classes=20,
                mode='inference',
                l2_regularization=0.0005,
                scales=[0.07, 0.15, 0.3, 0.45, 0.6, 0.75, 0.9, 1.05], # The scales for MS COCO are [0.04, 0.1, 0.26, 0.42, 0.58, 0.74, 0.9, 1.06]
                aspect_ratios_per_layer=[[1.0, 2.0, 0.5],
                                         [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                                         [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                                         [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                                         [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                                         [1.0, 2.0, 0.5],
                                         [1.0, 2.0, 0.5]],
               two_boxes_for_ar1=True,
               steps=[8, 16, 32, 64, 128, 256, 512],
               offsets=[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5],
               clip_boxes=False,
               variances=[0.1, 0.1, 0.2, 0.2],
               normalize_coords=True,
               subtract_mean=[123, 117, 104],
               swap_channels=False,
               confidence_thresh=0.5,
               iou_threshold=0.45,
               top_k=200,
               nms_max_output_size=400)

    # 2: Load the trained weights into the model.
    # TODO: Set the path of the trained weights.
    model.load_weights(model_path, by_name=True)

    # 3: Compile the model so that Keras won't complain the next time you load it.

    adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

    ssd_loss = SSDLoss(neg_pos_ratio=3, alpha=1.0)

    model.compile(optimizer=adam, loss=ssd_loss.compute_loss)
    
    return model

In [119]:
def load_graph(frozen_graph_filename):
    '''
    Load a frozen graph
    frozen_grapth_filename: path in the disk
    return: tensorflow graph
    '''
    # We load the protobuf file from the disk and parse it to retrieve the 
    # unserialized graph_def
    with tf.gfile.GFile(frozen_graph_filename, "rb") as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
        
    # Then, we import the graph_def into a new Graph and returns it 
    with tf.Graph().as_default() as graph:
        # The name var will prefix every op/nodes in your graph
        # Since we load everything in a new graph, this is not needed
        tf.import_graph_def(graph_def, name="prefix")
    return graph

def save_graph(name):
    '''
    Save grapth to load with tensorboard
    '''
    graph = load_graph(name)

    # save data from a grapth to display on tensorboard
    # tensorboard --logdir=wholepath
    # the path has to be absolute
    with tf.Session(graph=graph) as sess:
        tf.summary.FileWriter('../logs_ssd', sess.graph)

def show_operations_graph(graph):
    '''
    Print operations contained in a graph
    '''
    for op in graph.get_operations():
        print(op.name)

In [79]:
# Load models and weights
model_pascal_voc_07_plus_12_300x300 = '../weights/pascal_voc/pascal_voc_07_plus_12_300x300.h5'
model = load_model(model_pascal_voc_07_plus_12_300x300, 300, 300)

In [80]:
# Save models as .pbtxt or pb
tf.train.write_graph(K.get_session().graph, path_models, "ssd_model.pbtxt", as_text=True)
tf.train.write_graph(K.get_session().graph, path_models, "ssd_model.pb", as_text=False)

'../models/ssd_model.pb'

In [91]:
save_graph(path_models + '/ssd_model.pb')

In [81]:
# Add ops to save and restore all the variables.
saver = tf.train.Saver()
sess = keras.backend.get_session()
save_path = saver.save(sess, path_models + "/ssd_model.ckpt")

## Freeze the Graph
In order to convert the graph to tflite we need to freeze it, in this case we'll use the tools provided by tensorflow to do so.

We need to specify the graph, the checkpoints, the output name.

In [95]:
# Freeze the graph
import tensorflow as tf
from tensorflow.python.tools import freeze_graph

input_graph_path = path_models + '/ssd_model.pb'
checkpoint_path = path_models + '/ssd_model.ckpt'
input_saver_def_path = ""
input_binary = True
output_node_names = "decoded_predictions/loop_over_batch/TensorArrayStack/TensorArrayGatherV3"
restore_op_name = "save/restore_all"
filename_tensor_name = "save/Const:0"
output_frozen_graph_name = path_models + '/ssd_frozen_model.pb'
clear_devices = True


freeze_graph.freeze_graph(input_graph_path, input_saver_def_path,
                          input_binary, checkpoint_path, output_node_names,
                          restore_op_name, filename_tensor_name,
                          output_frozen_graph_name, clear_devices, "")

INFO:tensorflow:Restoring parameters from ../models/ssd_model.ckpt
INFO:tensorflow:Froze 71 variables.
INFO:tensorflow:Converted 71 variables to const ops.


## Optimize for inference

Optimize for inference will help us to remove those parts of the graph that were used during training.

We need the input and output names and the frozen model.

In [10]:
!python3 -m tensorflow.python.tools.optimize_for_inference \
--input=./extra_files/models/ssd_frozen_model.pb \
--output=./extra_files/models/ssd_opt_model.pb \
--frozen_graph=True --input_names=input_1 \
--output_names=predictions/concat

In [120]:
# summary model
# bazel build tensorflow/tools/graph_transforms:summarize_graph && \
# bazel-bin/tensorflow/tools/graph_transforms/summarize_graph \
# --in_graph=../ssd/extra_files/models/ssd_opt_model.pb

# Convert to tflite model
Finally we conver the model to tflite format, we need a frozen model in the same way the input and output name are required, and all the operations in the model need to be conpatible with tflite.

In [122]:
graph_def_file = "/home/aldopedraza/Documentos/models/ssd_frozen_model.pb"
input_arrays = ["input_1"]
output_arrays = ["decoded_predictions/loop_over_batch/TensorArrayStack/TensorArrayGatherV3"]

converter = tf.contrib.lite.TocoConverter.from_frozen_graph(
  graph_def_file, input_arrays, output_arrays, input_shapes={"input_1" : [1, 300, 300, 3]})
tflite_model = converter.convert()
open("/home/aldopedraza/Documentos/models/converted_model.tflite", "wb").write(tflite_model)

In [123]:
# !toco \
#   --graph_def_file=/home/aldopedraza/Documentos/ssd/extra_files/models/ssd_opt_model.pb \
#   --output_file=/home/aldopedraza/Documentos/ssd/extra_files/models/tflite_model.tflite \
#   --input_format=TENSORFLOW_GRAPHDEF --output_format=TFLITE \
#   --inference_type=QUANTIZED_UINT8 \
#   --input_shape="1,512,512,3" \
#   --input_array=input_1 \
#   --output_array=predictions/concat \
#   --std_dev_values=127.5 --mean_value=127.5

!toco \
--output_file=/home/aldopedraza/Documentos/ssd/extra_files/models/tflite_model.tflite \
--graph_def_file=/home/aldopedraza/Documentos/ssd/extra_files/models/ssd_opt_model.pb \
--output_format=TFLITE \
--inference_type=QUANTIZED_UINT8 \
--inference_input_type=FLOAT \
--input_arrays=input_1 \
--output_arrays=predictions/concat \
--input_shapes=1,512,512,3 \

## Test model to see if it's compatible with tflite

In order to be sure that the model works correctly what we can do it's to use the interpreter to load the model and test it.

In [1]:
import numpy as np
import tensorflow as tf
from scipy import misc

# Read image
image_test = misc.imread('./image_test.jpg')
image_test = misc.imresize(image_test, size=(300, 300))
misc.imsave('./resize_image.jpg', image_test)
image_test = image_test.reshape(1, 300, 300, 3)

# Load TFLite model and allocate tensors.
# interpreter = tf.contrib.lite.Interpreter(model_path="/home/aldopedraza/Documentos/ssd/extra_files/models/converted_model.tflite")
interpreter = tf.contrib.lite.Interpreter(model_path="../models/converted_model.tflite")

interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Test model on random input data.
input_shape = input_details[0]['shape']
input_data = np.array(image_test, dtype=np.float32)
interpreter.set_tensor(input_details[0]['index'], input_data)

interpreter.invoke()
output_data = interpreter.get_tensor(output_details[0]['index'])
print(output_data[0])

/home/aldopedraza/anaconda3/envs/tensorflow2/lib/python3.6/site-packages/ipykernel/__main__.py:6: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/home/aldopedraza/anaconda3/envs/tensorflow2/lib/python3.6/site-packages/ipykernel/__main__.py:7: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
/home/aldopedraza/anaconda3/envs/tensorflow2/lib/python3.6/site-packages/ipykernel/__main__.py:8: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.


[[9.9056888e-01 2.6890449e-04 2.9046461e-04 ... 1.0000000e-01
  2.0000000e-01 2.0000000e-01]
 [9.9315298e-01 1.7266681e-04 2.5180369e-04 ... 1.0000000e-01
  2.0000000e-01 2.0000000e-01]
 [9.9203098e-01 2.7664716e-04 2.3912858e-04 ... 1.0000000e-01
  2.0000000e-01 2.0000000e-01]
 ...
 [9.9938989e-01 4.6993050e-06 3.1778568e-06 ... 1.0000000e-01
  2.0000000e-01 2.0000000e-01]
 [9.9897754e-01 1.0331698e-05 5.4132288e-06 ... 1.0000000e-01
  2.0000000e-01 2.0000000e-01]
 [9.9964786e-01 5.0593994e-06 2.8518134e-06 ... 1.0000000e-01
  2.0000000e-01 2.0000000e-01]]


In [5]:
count = 0
for value in output_data[0]:
    if(value[15] > 0.5):
        print(value[15], " " + str(count))
    count += 1

0.6810785  6639
0.8500837  6753
0.6522881  6867
0.6811034  8223
0.7581048  8283


# Compare cell phone output and computer output

In [6]:
with open('./results.txt') as f:
    text = f.read()

In [7]:
# The output from the cellphone is store in a string
text2 = text.split(';')
list1 = []
for line in text.split(';'):
    list2 = []
    for j in line.split(','):
        list2.append(float(j))
    list1.append(list2)

In [8]:
output_computer = list(output_data.tolist())

In [10]:
import numpy as np
com = np.array(output_computer)
cell = np.array(list1)

In [19]:
cell[6639][15]

0.00565358

In [21]:
com[0][6639][15]

0.6810784935951233

In [20]:
cell[0][15]

0.0015088855

In [157]:
cell[0][21:25]

array([ 0.11123182,  0.24621835, -3.7149367 , -3.3556237 ])

In [159]:
cell[0][25: 29]

array([0.01333333, 0.01333333, 0.1       , 0.1       ])

In [160]:
cell[0][29:]

array([0.1, 0.1, 0.2, 0.2])

In [146]:
max_value = 0
i = 0
index = 0
for item in cell:
    max_item = item.max()
    if max_item > max_value:
        max_value = max_item
        index = i
    i += 1

In [148]:
index

2823

In [49]:
from scipy.spatial import distance

In [114]:
distance_acc = 0
for i in range(len(cell)):
    distance_acc += distance.euclidean(list1[i], output_computer[0][i])
    print(distance_acc)
    break

1.3140071604545618


In [115]:
# mean error with euclidean distance
distance_acc / 8732

0.0001504818094886122

In [116]:
from sklearn.metrics import mean_squared_error
error = 0
for i in range(len(cell)):
    error += mean_squared_error(list1[i], output_computer[0][i])
    print(error)
    break

0.052321661143207895


In [117]:
# mean error with mean square error
error / 8732

5.99194470261199e-06